<a href="https://colab.research.google.com/github/angelaguzmanar/Marketing/blob/main/Modelos_M2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3156241 sha256=f7be7fe8c3cb9d62e0a41f98d5ac2905fc08c56ce299fa40310137b6feb7cad9
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [2]:
#conectamos con Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
import pandas as pd
import sqlite3 as sql
from sklearn.preprocessing import MinMaxScaler
from ipywidgets import interact ## para análisis interactivo
from sklearn import neighbors ### basado en contenido un solo producto consumido
import joblib
import surprise
from surprise import Reader, Dataset
from surprise.model_selection import cross_validate, GridSearchCV
from surprise import KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline
from surprise.model_selection import train_test_split

In [4]:
# Conecta a la base de datos SQLite
conn = sql.connect('/content/drive/MyDrive/Analitica3/Entrega_Marketing/db_movies')

cur=conn.cursor()

### para ver las tablas que hay en la base de datos
cur.execute("select name from sqlite_master where type='table' ")
cur.fetchall()

[('ratings',),
 ('moviess',),
 ('ratings_copia',),
 ('exploracion',),
 ('data_final',),
 ('movies_final',),
 ('ratings_final',),
 ('movies',),
 ('usuarios_sel',),
 ('movies_sel',),
 ('ratingssin5',),
 ('sel5_movies',),
 ('movies_final5',),
 ('ratings_final5',),
 ('df_final',),
 ('reco',)]

In [5]:
df_explo=pd.read_csv('https://raw.githubusercontent.com/angelaguzmanar/Marketing/main/df_final.csv')
df_movies=pd.read_csv('https://raw.githubusercontent.com/angelaguzmanar/Marketing/main/movies_dum.csv')
df_moviesdum2 =pd.read_csv('https://raw.githubusercontent.com/angelaguzmanar/Marketing/main/moviesdum2.csv')

In [6]:
# Conecta a la base de datos SQLite
conn = sql.connect('/content/drive/MyDrive/Analitica3/Entrega_Marketing/db_movies')

In [7]:
# Conecta a la base de datos SQLite
conn = sql.connect('/content/drive/MyDrive/Analitica3/Entrega_Marketing/db_movies')

cur=conn.cursor()

### para ver las tablas que hay en la base de datos
cur.execute("select name from sqlite_master where type='table' ")
cur.fetchall()


[('ratings',),
 ('moviess',),
 ('ratings_copia',),
 ('exploracion',),
 ('data_final',),
 ('movies_final',),
 ('ratings_final',),
 ('movies',),
 ('usuarios_sel',),
 ('movies_sel',),
 ('ratingssin5',),
 ('sel5_movies',),
 ('movies_final5',),
 ('ratings_final5',),
 ('df_final',),
 ('reco',)]

In [8]:
dfmovies= pd.read_sql("select * from movies_final5", conn) ### convertir resultados de una consulta en una tabla de pandas
dfmovies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
2706,179401,Jumanji: Welcome to the Jungle (2017),Action|Adventure|Children
2707,179819,Star Wars: The Last Jedi (2017),Action|Adventure|Fantasy|Sci-Fi
2708,180031,The Shape of Water (2017),Adventure|Drama|Fantasy
2709,183897,Isle of Dogs (2018),Animation|Comedy


In [9]:
dfratings= pd.read_sql("select * from ratings_final5", conn) ### convertir resultados de una consulta en una tabla de pandas
dfratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
66190,609,786,3.0,847221025
66191,609,828,3.0,847221054
66192,609,892,3.0,847221080
66193,609,1056,3.0,847221080


## Sistema 3

In [10]:
userId=10	### para ejemplo manual

###seleccionar solo los ratings del usuario seleccionado
rating =pd.read_sql('select * from ratings_final5 where userId=:user',conn, params={'user':userId})

    ###convertir ratings del usuario a array
l_movies_r=rating['movieId'].to_numpy()

    ### filtrar peliculas calificados por el usuario
movies_r=df_moviesdum2[df_moviesdum2['movieId'].isin(l_movies_r)]

    ## eliminar columna nombre pelicula y id
movies_r=movies_r.drop(columns=['movieId','title'])
movies_r["indice"]=1 ### para usar group by y que quede en formato pandas tabla de centroide
    ##centroide o perfil del usuario
centroide=movies_r.groupby("indice").mean()


    ### filtrar peliculas no vistas
movies_nr=df_moviesdum2[~df_moviesdum2['movieId'].isin(l_movies_r)]
    ## eliminbar nombre peli y ud
movies_nr=movies_nr.drop(columns=['movieId','title'])

    ### entrenar modelo
model=neighbors.NearestNeighbors(n_neighbors=11, metric='cosine')
model.fit(movies_nr)
dist, idlist = model.kneighbors(centroide)

ids=idlist[0] ### queda en un array anidado, para sacarlo
recomend_b=df_movies.loc[ids][['title','movieId']]
leidos=df_movies[df_movies['movieId'].isin(l_movies_r)][['title','movieId']]
recomend_b

,title,movieId
2575,Interstellar (2014),109487
2396,"Sorcerer's Apprentice, The (2010)",79139
2373,"Book of Eli, The (2010)",73321
2272,Iron Man (2008),59315
2224,Superbad (2007),54503
2227,3:10 to Yuma (2007),54997
2140,Click (2006),45672
2187,Music and Lyrics (2007),51084
1963,Dogville (2003),7371
1998,Fahrenheit 9/11 (2004),8622


In [11]:
rating =pd.read_sql('select * from ratings_final5',conn)
rating

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
66190,609,786,3.0,847221025
66191,609,828,3.0,847221054
66192,609,892,3.0,847221080
66193,609,1056,3.0,847221080


In [12]:
df_moviesdum2

,movieId,title,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,year_sc
0,1,Toy Story (1995),0,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0.765306
1,2,Jumanji (1995),0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0.765306
2,3,Grumpier Old Men (1995),0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0.765306
3,4,Waiting to Exhale (1995),0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0.765306
4,5,Father of the Bride Part II (1995),0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0.765306
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2706,179401,Jumanji: Welcome to the Jungle (2017),0,1,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0.989796
2707,179819,Star Wars: The Last Jedi (2017),0,1,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0.989796
2708,180031,The Shape of Water (2017),0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.989796
2709,183897,Isle of Dogs (2018),0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1.000000


In [13]:
df_movies.to_sql('df_movies', conn, index=False, if_exists='replace')

2711

In [14]:
df_moviesdum2.to_sql('df_moviesdum2', conn, index=False, if_exists='replace')

2711

In [15]:
def recomendar(userId=list(dfratings['userId'].value_counts().index)):

    ###seleccionar solo los ratings del usuario seleccionado
    rating =pd.read_sql('select * from ratings_final5 where userId=:user',conn, params={'user':userId})

    ###convertir ratings del usuario a array
    l_movies_r=rating['movieId'].to_numpy()

    ### filtrar peliculas calificados por el usuario
    movies_r=df_moviesdum2[df_moviesdum2['movieId'].isin(l_movies_r)]

    ## eliminar columna nombre pelicula y id
    movies_r=movies_r.drop(columns=['movieId','title'])
    movies_r["indice"]=1 ### para usar group by y que quede en formato pandas tabla de centroide
    ##centroide o perfil del usuario
    centroide=movies_r.groupby("indice").mean()


    ### filtrar peliculas no vistas
    movies_nr=df_moviesdum2[~df_moviesdum2['movieId'].isin(l_movies_r)]
    ## eliminbar nombre peli y id
    movies_nr=movies_nr.drop(columns=['movieId','title'])

    ### entrenar modelo
    model=neighbors.NearestNeighbors(n_neighbors=11, metric='cosine')
    model.fit(movies_nr)
    dist, idlist = model.kneighbors(centroide)

    ids=idlist[0] ### queda en un array anidado, para sacarlo
    recomend_b=df_movies.loc[ids][['title','movieId']]
    leidos=df_movies[df_movies['movieId'].isin(l_movies_r)][['title','movieId']]

    return recomend_b

print(interact(recomendar))

interactive(children=(Dropdown(description='userId', options=(590, 600, 483, 489, 19, 305, 438, 477, 217, 105,…

<function recomendar at 0x7dff9b1764d0>


In [16]:
userId=10	### para ejemplo manual

###seleccionar solo los ratings del usuario seleccionado
rating =pd.read_sql('select * from ratings_final5 where userId=:user',conn, params={'user':userId})

    ###convertir ratings del usuario a array
l_movies_r=rating['movieId'].to_numpy()

    ### filtrar peliculas calificados por el usuario
movies_r=df_moviesdum2[df_moviesdum2['movieId'].isin(l_movies_r)]

    ## eliminar nombre pelicula y id
movies_r=movies_r.drop(columns=['movieId','title'])
movies_r["indice"]=1 ### para usar group by y que quede en formato pandas tabla de centroide
    ##centroide o perfil del usuario
centroide=movies_r.groupby("indice").mean()


    ### filtrar peliculass no vistas
movies_nr=df_moviesdum2[~df_moviesdum2['movieId'].isin(l_movies_r)]
    ## eliminbar nombre pelicula y id
movies_nr=movies_nr.drop(columns=['movieId','title'])

    ### entrenar modelo
model=neighbors.NearestNeighbors(n_neighbors=11, metric='euclidean')
model.fit(movies_nr)
dist, idlist = model.kneighbors(centroide)

ids=idlist[0] ### queda en un array anidado, para sacarlo
recomend_b=df_movies.loc[ids][['title','movieId']]
leidos=df_movies[df_movies['movieId'].isin(l_movies_r)][['title','movieId']]
recomend_b

,title,movieId
1907,"Matrix Revolutions, The (2003)",6934
1918,"Cabinet of Dr. Caligari, The (Cabinet des Dr. ...",6987
1959,Secret Window (2004),7347
1910,Father of the Bride (1991),6944
1969,Kill Bill: Vol. 2 (2004),7438
2027,Saw (2004),8957
2033,National Treasure (2004),8972
1823,My Neighbor Totoro (Tonari no Totoro) (1988),5971
1800,Saturday Night Fever (1977),5693
1869,Pirates of the Caribbean: The Curse of the Bla...,6539


In [17]:
def recomendar(userId=list(dfratings['userId'].value_counts().index)):

    ###seleccionar solo los ratings del usuario seleccionado
    rating =pd.read_sql('select * from ratings_final5 where userId=:user',conn, params={'user':userId})

    ###convertir ratings del usuario a array
    l_movies_r=rating['movieId'].to_numpy()

    ### filtrar peliculas calificados por el usuario
    movies_r=df_moviesdum2[df_moviesdum2['movieId'].isin(l_movies_r)]

    ## eliminar columna nombre pelicula y id
    movies_r=movies_r.drop(columns=['movieId','title'])
    movies_r["indice"]=1 ### para usar group by y que quede en formato pandas tabla de centroide
    ##centroide o perfil del usuario
    centroide=movies_r.groupby("indice").mean()


    ### filtrar peliculas no vistas
    movies_nr=df_moviesdum2[~df_moviesdum2['movieId'].isin(l_movies_r)]
    ## eliminbar nombre pelicula y id
    movies_nr=movies_nr.drop(columns=['movieId','title'])

    ### entrenar modelo
    model=neighbors.NearestNeighbors(n_neighbors=11, metric='euclidean')
    model.fit(movies_nr)
    dist, idlist = model.kneighbors(centroide)

    ids=idlist[0] ### queda en un array anidado, para sacarlo
    recomend_b=df_movies.loc[ids][['title','movieId']]
    leidos=df_movies[df_movies['movieId'].isin(l_movies_r)][['title','movieId']]

    return recomend_b

print(interact(recomendar))

interactive(children=(Dropdown(description='userId', options=(590, 600, 483, 489, 19, 305, 438, 477, 217, 105,…

<function recomendar at 0x7dff99b25000>


## Sistema 4

In [18]:
dfratings=pd.read_sql('select * from ratings_final5 where rating>0', conn)
dfratings['rating'].unique()

array([4. , 5. , 3. , 2. , 1. , 4.5, 3.5, 2.5, 0.5, 1.5])

In [19]:
####los datos deben ser leidos en un formato espacial para surprise
reader = Reader(rating_scale=(0, 5)) ### la escala de la calificación
###las columnas deben estar en orden estándar: user item rating
data   = Dataset.load_from_df(dfratings[['userId','movieId','rating']], reader)


#####Existen varios modelos
models=[KNNBasic(),KNNWithMeans(),KNNWithZScore(),KNNBaseline()]
results = {}

In [20]:
#### función para probar varios modelos ##########

for model in models:

    CV_scores = cross_validate(model, data, measures=["MAE","RMSE"], cv=5, n_jobs=-1)

    result = pd.DataFrame.from_dict(CV_scores).mean(axis=0).\
             rename({'test_mae':'MAE', 'test_rmse': 'RMSE'})
    results[str(model).split("algorithms.")[1].split("object ")[0]] = result


performance_df = pd.DataFrame.from_dict(results).T
performance_df.sort_values(by='RMSE') #El mejor modelo es knns.KNNBaseline

,MAE,RMSE,fit_time,test_time
knns.KNNBaseline,0.667089,0.874449,0.268381,2.114429
knns.KNNWithZScore,0.670696,0.885146,0.377635,3.598164
knns.KNNWithMeans,0.676750,0.886335,0.376668,3.203504
knns.KNNBasic,0.722180,0.944498,0.133252,1.337893


In [21]:
help(surprise.similarities.pearson_baseline)

Help on cython_function_or_method in module surprise.similarities:

pearson_baseline(n_x, yr, min_support, global_mean, x_biases, y_biases, shrinkage=100.0)
    Compute the (shrunk) Pearson correlation coefficient between all pairs
    of users (or items) using baselines for centering instead of means.
    
    The shrinkage parameter helps to avoid overfitting when only few ratings
    are available (see :ref:`similarity_measures_configuration`).
    
    The Pearson-baseline correlation coefficient is defined as:
    
    .. math::
        \text{pearson_baseline_sim}(u, v) = \hat{\rho}_{uv} = \frac{
            \sum\limits_{i \in I_{uv}} (r_{ui} -  b_{ui}) \cdot (r_{vi} -
            b_{vi})} {\sqrt{\sum\limits_{i \in I_{uv}} (r_{ui} -  b_{ui})^2}
            \cdot \sqrt{\sum\limits_{i \in I_{uv}} (r_{vi} -  b_{vi})^2}}
    
    or
    
    .. math::
        \text{pearson_baseline_sim}(i, j) = \hat{\rho}_{ij} = \frac{
            \sum\limits_{u \in U_{ij}} (r_{ui} -  b_{ui}) \cdot (r

In [22]:
param_grid = {'sim_options' : {'metric': ['msd','cosine', 'pearson'], \
                                'min_support': [5], \
                                'user_based': [False, True], \
                                'shrinkage': [10, 50,100,150]
}}

In [23]:
gridsearchKNNBaseline = GridSearchCV(KNNBaseline, param_grid, measures=['rmse'], \
                                      cv=2, n_jobs=-1)

In [24]:
gridsearchKNNBaseline.fit(data)

In [25]:
gridsearchKNNBaseline.best_params["rmse"]

{'sim_options': {'metric': 'msd',
  'min_support': 5,
  'user_based': True,
  'shrinkage': 10}}

In [26]:
gridsearchKNNBaseline.best_score["rmse"]

0.9101551658883222

In [27]:
gs_model=gridsearchKNNBaseline.best_estimator['rmse'] ### mejor estimador de gridsearch

In [28]:
################# Entrenar con todos los datos y Realizar predicciones con el modelo afinado

trainset = data.build_full_trainset()
model=gs_model.fit(trainset) ## se reentrena sobre todos los datos posibles (sin dividir)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.


In [29]:
predset = trainset.build_anti_testset() ### crea una tabla con todos los usuarios y las peliculas que no han visto
len(predset)

1530584

In [30]:
predictions = gs_model.test(predset)

In [31]:
predictions_df = pd.DataFrame(predictions)

(1530584, 5)

In [32]:
predictions_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1530584 entries, 0 to 1530583
Data columns (total 5 columns):
 #   Column   Non-Null Count    Dtype  
---  ------   --------------    -----  
 0   uid      1530584 non-null  int64  
 1   iid      1530584 non-null  int64  
 2   r_ui     1530584 non-null  float64
 3   est      1530584 non-null  float64
 4   details  1530584 non-null  object 
dtypes: float64(2), int64(2), object(1)
memory usage: 58.4+ MB


In [33]:
predictions_df['r_ui'].unique() ### promedio de ratings

array([3.62207115])

In [36]:
##### funcion para recomendar las peliculas con mejores predicciones y llevar base de datos para consultar resto de información
def recomendaciones(userId=list(dfratings['userId'].value_counts().index), n_r = 10):

    predictions_userID = predictions_df[predictions_df['uid'] == userId].\
                    sort_values(by="est", ascending = False).head(n_r)

    recomendados = predictions_userID[['iid','est']]
    recomendados.to_sql('reco',conn,if_exists="replace")

    recomendados=pd.read_sql('''select a.*, b.title
                             from reco a left join movies_final5 b
                             on a.iid=b.movieId ''', conn)

    return(recomendados)
print(interact(recomendaciones))

interactive(children=(Dropdown(description='userId', options=(590, 600, 483, 489, 19, 305, 438, 477, 217, 105,…

<function recomendaciones at 0x7dff943c75b0>


In [37]:
recomendaciones(userId=381,n_r=10)

,index,iid,est,title
0,959647,177593,4.813418,"Three Billboards Outside Ebbing, Missouri (2017)"
1,960509,6460,4.778690,"Trial, The (Procès, Le) (1962)"
2,960908,2239,4.745599,Swept Away (Travolti da un insolito destino ne...
3,960971,4642,4.628456,Hedwig and the Angry Inch (2000)
4,960788,1178,4.621437,Paths of Glory (1957)
5,960945,2436,4.587482,Tea with Mussolini (1999)
6,960699,1683,4.578290,"Wings of the Dove, The (1997)"
7,958772,940,4.566752,"Adventures of Robin Hood, The (1938)"
8,960536,158966,4.564028,Captain Fantastic (2016)
9,960570,5690,4.563292,Grave of the Fireflies (Hotaru no haka) (1988)
